<a href="https://colab.research.google.com/github/mukeshnarendran7/ML_DL/blob/master/collab_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook

In [2]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [3]:
path.ls()

(#23) [Path('/root/.fastai/data/ml-100k/u.genre'),Path('/root/.fastai/data/ml-100k/u.info'),Path('/root/.fastai/data/ml-100k/u.item'),Path('/root/.fastai/data/ml-100k/u1.test'),Path('/root/.fastai/data/ml-100k/u1.base'),Path('/root/.fastai/data/ml-100k/mku.sh'),Path('/root/.fastai/data/ml-100k/ua.test'),Path('/root/.fastai/data/ml-100k/u3.test'),Path('/root/.fastai/data/ml-100k/u2.test'),Path('/root/.fastai/data/ml-100k/allbut.pl')...]

In [20]:
ratings = pd.read_csv(path/'u.data', delimiter = '\t', header = None,
                      names = ['user', 'movie', 'rating', 'timestamp'])
ratings.head(13)

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [29]:
a = [i for i in ratings['user']][:10]
b = [i for i in ratings['movie']][:10]

In [9]:
n_users = 10
n_movies = 20
# list the ids of the most frequent users (those who rated the most movies)
most_frequent_users = list(ratings.user.value_counts().index[:n_users])
denser_ratings = ratings[ratings.user.isin(most_frequent_users)]

# list the ids of the most frequent movies within this group of users
dense_users_most_rated = list(denser_ratings.movie.value_counts().index[:n_movies])
# grab all the most frequent movies rated by the most frequent users
denser_movies = ratings[ratings.movie.isin(dense_users_most_rated)]

# plot the crosstab
pd.crosstab(denser_users.user, denser_movies.movie, values=ratings.rating, aggfunc='mean').fillna('-')

movie,4,22,50,69,70,82,88,96,97,172,202,204,210,318,385,423,443,550,655,739
user,,,,,,,,,,,,,,,,,,,,
13,5,4,5,4,3,2,4,4,4,5,5,5,3,3,3,5,4,4,5,4
234,4,4,4,4,3,3,3,2,2,3,3,2,3,4,2,4,3,2,3,3
276,4,5,5,4,4,4,3,5,3,5,4,5,4,5,4,5,4,4,4,2
303,4,5,5,5,4,4,4,5,5,5,5,4,4,5,4,4,4,3,5,5
393,4,4,5,4,3,4,3,4,4,5,3,4,4,3,4,3,3,3,3,3
405,4,5,5,4,3,4,3,3,2,5,4,5,5,5,1,5,4,2,5,2
416,4,5,5,4,5,5,3,4,5,5,4,5,5,5,5,4,5,4,5,5
450,3,5,5,4,4,3,5,4,4,4,4,4,3,5,4,5,4,4,4,4
537,2,2,4,2,4,2,2,3,2,3,3,3,3,4,2,2,3,2,3,1


In [21]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [23]:
ratings = ratings.merge(movies)
ratings.head(5)

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [24]:
dls = CollabDataLoaders.from_df(ratings, item_name = 'title', bs = 64)
dls.show_batch()

,user,title,rating
0,542,My Left Foot (1989),4
1,422,Event Horizon (1997),3
2,311,"African Queen, The (1951)",4
3,595,Face/Off (1997),4
4,617,Evil Dead II (1987),1
5,158,Jurassic Park (1993),5
6,836,Chasing Amy (1997),3
7,474,Emma (1996),3
8,466,Jackie Chan's First Strike (1996),3
9,554,Scream (1996),3


In [25]:
dls.classes

{'title': ['#na#', "'Til There Was You (1997)", '1-900 (1994)', '101 Dalmatians (1996)', '12 Angry Men (1957)', '187 (1997)', '2 Days in the Valley (1996)', '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)', '3 Ninjas: High Noon At Mega Mountain (1998)', '39 Steps, The (1935)', '8 1/2 (1963)', '8 Heads in a Duffel Bag (1997)', '8 Seconds (1994)', 'A Chef in Love (1996)', 'Above the Rim (1994)', 'Absolute Power (1997)', 'Abyss, The (1989)', 'Ace Ventura: Pet Detective (1994)', 'Ace Ventura: When Nature Calls (1995)', 'Across the Sea of Time (1995)', 'Addams Family Values (1993)', 'Addicted to Love (1997)', 'Addiction, The (1995)', 'Adventures of Pinocchio, The (1996)', 'Adventures of Priscilla, Queen of the Desert, The (1994)', 'Adventures of Robin Hood, The (1938)', 'Affair to Remember, An (1957)', 'African Queen, The (1951)', 'Afterglow (1997)', 'Age of Innocence, The (1993)', 'Aiqing wansui (1994)', 'Air Bud (1997)', 'Air Force One (1997)', 'Air Up There, The (199

In [31]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
print(f"length of nusers: {n_users}  and n_movies: {n_movies}")
n_factors = 5
user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors) 
print(f"length of userfactors: {user_factors.shape}  and movie_factors: {movie_factors.shape}")

length of nusers: 944  and n_movies: 1665
length of userfactors: torch.Size([944, 5])  and movie_factors: torch.Size([1665, 5])


In [39]:
one_hot3 = one_hot(3, n_users).float()
user_factors.t()@ one_hot3

tensor([-0.7443,  0.1431,  0.0563,  0.8255, -1.5452])

In [37]:
user_factors[3]

tensor([-0.7443,  0.1431,  0.0563,  0.8255, -1.5452])

In [53]:
class DotProduct(Module):
  def __init__(self, n_users, n_movies, n_factors):
    self.user_factors = (n_users, n_factors)
    self.movie_factors = (n_movies, n_factors)

  def forward(self, x):
    users = self.user_factors(x[:,0])
    movies = self.movie_factors(x[:,1])
    return (users*movies).sum(dim=1)

In [54]:
x, y = dls.one_batch()
x.shape, y.shape

(torch.Size([64, 2]), torch.Size([64, 1]))

In [55]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func= MSELossFlat())

In [56]:
learn.fit_one_cycle(5)

IndexError: ignored